# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [ ]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [ ]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [ ]:
# Import the necessary CSVs to Pandas DataFrames
# YOUR CODE HERE!

csv_path1= Path("Data/neighborhoods_coordinates.csv")
csv_path2 = Path("Data/sfo_neighborhoods_census_data.csv")

sfo_data = pd.read_csv(csv_path2, index_col="year")
neighborhood_df = pd.read_csv(csv_path1)

sfo_data.head()


In [ ]:
neighborhood_df.head()


- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [ ]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    
    housing_units = (sfo_data[['housing_units']].groupby('year').mean())
    min = housing_units.min()['housing_units']
    max = housing_units.max()['housing_units']
    
    housing_plot = housing_units.hvplot.bar(ylim = (min-2000, max+2000),title="Housing Units in SF Avg per Year",figsize=(12,8))

    housing_pane = pn.panel(housing_plot)

    return housing_plot


    # YOUR CODE HERE!


def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    
    avg_gross_rent = sfo_data[['gross_rent']].groupby('year').mean()

    gross_rent_plot = avg_gross_rent.hvplot.line(title="Average Gross Rent per Year",figsize=(12,8))
    
    gross_rent_pane = pn.panel(gross_rent_plot)
    
    return gross_rent_plot

    # YOUR CODE HERE!




def average_sales_price():
    """Average Sales Price Per Year."""
    
    avg_sqft_price = sfo_data[['sale_price_sqr_foot']].groupby('year').mean()

    avg_sqt_ft_plot = avg_sqft_price.hvplot.line(title= "Average Sale Price Square Foot per Year", figsize = (12,8))
    
    sales_price_pane = pn.panel(avg_sqt_ft_plot)
    
    return avg_sqt_ft_plot



def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    
    grouped_neighborhood = sfo_data.groupby([sfo_data.index, "neighborhood"]).mean()
    grouped_price = grouped_neighborhood['sale_price_sqr_foot']
    grouped_df = pd.DataFrame(grouped_price).reset_index()
    grouped_df.head()

    
    avg_price_neighborhood_plot = grouped_df.hvplot.line(
    "year",
    "sale_price_sqr_foot",
    xlabel= "Year",
    ylabel="Average Sale Price per Square Foot",
    groupby="neighborhood",
    )
    
    price_by_neighborhood_pane = pn.panel(avg_price_neighborhood_plot)
    
    return avg_price_neighborhood_plot
    # YOUR CODE HERE!



def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""

    df_expensive = sfo_data.groupby("neighborhood").mean()
    df_expensive = df_expensive.sort_values("sale_price_sqr_foot", ascending=False).head(10)
    df_expensive = df_expensive.reset_index()
    df_expensive
    
    top_expensive_plot = df_expensive.hvplot.bar(
    x ="neighborhood",
    y ="sale_price_sqr_foot",
    title="Top 10 Expensive Neighborhoods in San Francisco",
    xlabel="Neighborhood",
    ylabel="Average Sale Price per Square Foot",
    height = 400,
    rot = 90,
    )
   
    top_expensive_pane = pn.panel(top_expensive_plot)
    
    return top_expensive_plot
    
    # YOUR CODE HERE!



def most_expensive_neighborhoods_rent_sales():
    """Comparison of Rent and Sales Prices of Most Expensive Neighborhoods."""   
    
    df_expensive = sfo_data.groupby("neighborhood").mean()
    df_expensive = df_expensive.sort_values("sale_price_sqr_foot", ascending=False).head(10)
    df_expensive = df_expensive.reset_index()

    expensive_neighorhood = sfo_data[(sfo_data['neighborhood'].isin(df_expensive['neighborhood']))]
    
    

    stacked_graphs = expensive_neighorhood.hvplot.bar( x = 'year', y = ['sale_price_sqr_foot', 'gross_rent'], xlabel = 'Year', groupby = 'neighborhood', rot = 90, height = 400)

    

    return stacked_graphs

    


    
    
def parallel_coordinates():
    """Parallel Coordinates Plot."""

    


def parallel_categories():
    """Parallel Categories Plot."""
    
    # YOUR CODE HERE!



def neighborhood_map():
    """Neighborhood Map."""

neighborhood_mean = sfo_data.groupby("neighborhood").mean()
    neighborhood_mean.reset_index(inplace=True)
    
    
    avg_loc_neighborhood = pd.concat([neighborhood_df,
                        neighborhood_mean['sale_price_sqr_foot'],
                        neighborhood_mean['housing_units'],
                        neighborhood_mean['gross_rent']
                        ], axis=1).dropna()
    
    
    px.set_mapbox_access_token(map_box_api)

    
    map_plot = px.scatter_mapbox(
    avg_loc_neighborhood,
    lat="Lat",
    lon="Lon",
    size="sale_price_sqr_foot",
    color="gross_rent",
    title="Average Sale Price per Square Foot and Gross Rent in San Francisco"
    )
    
    return map_plot


def sunburst():
    """Sunburst Plot."""
    
    # YOUR CODE HERE!


## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [ ]:
# Create a Title for the Dashboard

title = pn.pane.Markdown(
    """
    # Real Estate Data in San Francisco from 2010 to 2016
    """,
    width = 600,
)



# Create a tab layout for the dashboard
# YOUR CODE HERE!

tabs = pn.Tabs(
    ('Welcome', pn.Column(title, neighborhood_map())),
    ('Average Rent and Sale Price 2010-2016', pn.Column(housing_units_per_year() ,average_gross_rent(), average_sales_price())),
    ('Average Sale Price by Neighborhood', pn.Column(average_price_by_neighborhood(), )),
    ('Most Expensive Neighborhoods', pn.Column(top_most_expensive_neighborhoods(), most_expensive_neighborhoods_rent_sales())))
    
panel = pn.Column(title, tabs, width = 800)


# Create the dashboard
# YOUR CODE HERE!

panel.servable()


## Serve the Panel Dashboard

In [ ]:
# Serve the# dashboard
# YOUR CODE HERE!

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [ ]:
housing_units_per_year()

In [ ]:
average_gross_rent()

In [ ]:
average_sales_price()

In [ ]:
average_price_by_neighborhood()

In [ ]:
top_most_expensive_neighborhoods()

In [ ]:
most_expensive_neighborhoods_rent_sales()

In [ ]:
neighborhood_map().show()

In [ ]:
# parallel_categories()

In [ ]:
# parallel_coordinates()

In [ ]:
# sunburst()